In [16]:
!nvidia-smi

Mon Jun 14 19:27:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650    Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8     2W /  N/A |    614MiB /  3911MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
#Importar librerías a utilizar
import cudf, cuml, cuxfilter
from pandas.io.json import json_normalize 
import pandas as pd

In [18]:
#Fase de extracción y lectura con Rapids
import time

#extracción y lectura con Pandas
pd_PandasBig = pd.read_csv('dataOpenCampus2.csv')

startEx = time.time()

#extraccion y lectura con cudf
gdfBig = cudf.read_csv('dataOpenCampus2.csv')

endExt = time.time()
caculatedCUDFBig = endExt - startEx

print("Cálculo de extracción y lectura con cuDF = {}".format(caculatedCUDFBig))

Cálculo de extracción y lectura con cuDF = 1.0622437000274658


In [20]:
print('Descripción de la extracción y lectura del archivo:')
gdfBig.head()

Descripción de la extracción y lectura del archivo:


,u,n,o
0,/edx/var/log/tracking/tracking.log-20171110-15...,<NA>,<NA>
1,/edx/var/log/tracking/tracking.log-20171110-15...,<NA>,<NA>
2,/edx/var/log/tracking/tracking.log-20171110-15...,<NA>,<NA>
3,/edx/var/log/tracking/tracking.log-20171110-15...,<NA>,<NA>
4,/edx/var/log/tracking/tracking.log-20171110-15...,<NA>,<NA>


In [23]:
import json

cudf_Pandas = pd_PandasBig

start = time.time()

#crear formato json para despues crear un dataframe nuevo
start = time.time()
lista = list(cudf_Pandas['u'])

a = []
for i in range(0,len(lista)):
    d = lista[i].split('.gz:')
    a.append(json.loads((d[1])))
    
#Crear DataFrame con Pandas
pdPandas = pd.DataFrame(a)

# Convierto todo el dataframe a str
nuevo_pdPandas = pdPandas.applymap(str)

#solo conservo los datos de la columna username
#mayor a 0
snNull = nuevo_pdPandas[nuevo_pdPandas.username.map(len) > 0]

#elimino username dicc y opencampus
snDicc = snNull[snNull.username != 'dicc']
snCampus = snDicc[snDicc.username != 'opencampus']

#convertir de pandas DataFrame
#aún cudf DataFrame
gdf_new = cudf.DataFrame.from_pandas(snCampus)

gdf_new.drop(['accept_language', 'name', 'agent', 'page', 'session',
              'event', 'event_type'], axis=1)

#eliminar filas con los datos perdidos
gdf_new.dropna(axis=0, how='any')
gdf_new.head(2)

language = gdf_new['accept_language'].str.split(',', n=4, expand=True)
langNew = language.drop([1,2,3,4], axis=1)

langNew.columns = ['language']
newData = gdf_new.join(langNew, how='left')
newData = newData.drop(['accept_language','agent','page','session',
                        'context','event','event_type'], axis=1)

#eliminar filas con los datos perdidos
gdf_newData = newData.dropna(axis=0, how='any')

#dividir columna time
vtime = gdf_newData['time']
vtime.str.normalize_spaces()

new_time = vtime.str.split(pat = "-", n=2, expand=True)

day = new_time[2]
daytime = day.str.split(pat = 'T', n=1, expand=True)
daytime.drop(1, axis=1)
daytime.columns = ['day', 'hour']

timenew = new_time.join(daytime, how='left')
timenew.columns = ['year','month','d','day','hour']
timenew = timenew.drop(['d'], axis=1)
timenew

#Unir todo al dataframe time al general
newData = newData.join(timenew, how='left')
newDataMejorada = newData.drop(['time'], axis=1)

#eliminar filas con datos perdidos
newDataMejorada.dropna(axis=0, how='any')

#resetear index
newDataMejorada = newDataMejorada.reset_index()
DataMejorada = newDataMejorada.drop(['index'], axis=1)

end = time.time()
calculo_Transform= end - start

print('Cálculo de transformación con cuDF:', calculo_Transform)

Cálculo de transformación con cuDF: 14.276833772659302


In [24]:
#Exploración de todo el Dataset con cudf
print('Nombre de columnas:',DataMejorada.columns)
print('Dimension DataFrame:',DataMejorada.shape)
print('Tipos de datos de cada columna', DataMejorada.dtypes)
print('Información DATA general con Rapids:')
DataMejorada.head(2)

Nombre de columnas: Index(['username', 'event_source', 'name', 'host', 'referer', 'ip', 'language',
       'year', 'month', 'day', 'hour'],
      dtype='object')
Dimension DataFrame: (392872, 11)
Tipos de datos de cada columna username        object
event_source    object
name            object
host            object
referer         object
ip              object
language        object
year            object
month           object
day             object
hour            object
dtype: object
Información DATA general con Rapids:


,username,event_source,name,host,referer,ip,language,year,month,day,hour
0,esteban14cueva,server,nan,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,201.219.63.99,es-419,2017,12,11,01:15:18.724071+00:00
1,esteban14cueva,server,nan,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,201.219.63.99,es-419,2017,12,11,01:15:56.298750+00:00


In [25]:
#Cargar la data en un nuevo csv
start = time.time()

da_data = 'dataRepresentativaBig.csv'
DataMejorada.to_csv(da_data)

end = time.time()
calculated = end - start

print("Cálculo carga con cuDF = {}".format(calculated))

Cálculo carga con cuDF = 0.6893894672393799


In [26]:
#Fase dividir Dataset
from cuml.preprocessing.model_selection import train_test_split

#Asignar dataset general a nuevas variables
gdf_dataX = DataMejorada
gdf_dataY = DataMejorada

#Dividir dataset en train y test
#datatrain 80% y datatest 20%
X_train, X_test, y_train, y_test = train_test_split(gdf_dataX, gdf_dataY,
                                                    train_size=0.8)
#información del dataset dividido
print(f'Dataset Original: {DataMejorada.shape[0], gdf_dataY.shape[0]} elementos')
print(f'Data X_train: {X_train.shape[0]} elements')
print(f'Data X_test: {X_test.shape[0]} elements')
print(f'Data y_train: {y_train.shape[0]} elements')
print(f'Data y_test: {y_test.shape[0]} elements')

Dataset Original: (392872, 392872) elementos
Data X_train: 314297 elements
Data X_test: 78575 elements
Data y_train: 314297 elements
Data y_test: 78575 elements


In [27]:
#Fase Preprocesamiento cuml
import time
from cuml.preprocessing.LabelEncoder import LabelEncoder

startPre = time.time()

le = LabelEncoder()

#Convertir a variables categóricas
#seleccionar columnas potenciales para train de xTrain
value_usernameX = le.fit_transform(X_train.username)
value_EventSourceX = le.fit_transform(X_train.event_source)
value_nameX = le.fit_transform(X_train.name)
value_DayX = le.fit_transform(X_train.day)

#seleccionar columnas potenciales para test de xTest
value_usernameXT = le.fit_transform(X_test.username)
value_EventSourceXT = le.fit_transform(X_test.event_source)
value_nameXT = le.fit_transform(X_test.name)

#seleccionar columnas potenciales para train de yTrain
value_usernameY = le.fit_transform(y_train.username)
value_EventSourceY = le.fit_transform(y_train.event_source)
value_nameY = le.fit_transform(y_train.name)

#seleccionar dos columnas potenciales para test de yTest
value_usernameYT = le.fit_transform(y_test.username)
value_EventSourceYT = le.fit_transform(y_test.event_source)
value_nameYT = le.fit_transform(y_test.name)

endPre = time.time()
calculatedPre = endPre - startPre

print("Cálculo preprocesamiento con cuML = {}".format(calculatedPre))

Cálculo preprocesamiento con cuML = 0.31316566467285156


In [30]:
#Fase Uso de los algoritmos ML de cuml: Algoritmo LinearRegression
import cupy as cp
from cuml.linear_model import LinearRegression

#Creación de un nuevo dataframe Train 
dataTrain = cudf.DataFrame()
dataTrain['username'] = cp.asarray(value_usernameX, dtype=cp.float32)
dataTrain['day'] = cp.asarray(value_DayX, dtype=cp.float32)

#Creación de un nuevo dataframe Test
new_value = cp.asarray(value_usernameY, dtype=cp.float32)
dataTest = cudf.Series(new_value)

startRli = time.time()

#Entrenando el algoritmo
lr = LinearRegression(fit_intercept = True, normalize = False,algorithm = "eig")
reg = lr.fit(dataTrain, dataTest)

endRli = time.time()
calculatedRLi = endRli - startRli

print("Algoritmo Regresión Lineal")
print("Coeficientes:")
print(reg.coef_)
print('')
print("Intercepción:")
print(reg.intercept_)
print('')
print('Predicción del modelo:')
print(lr.predict(dataTrain))

print("Cálculo modelo de entrenamiento algoritmo Regresion Lineal de cuML = {}".format(calculatedRLi))

Algoritmo Regresión Lineal
Coeficientes:
0    1.000002e+00
1    9.069567e-07
dtype: float32

Intercepción:
-0.000701904296875

Predicción del modelo:
0         248.999847
1         377.000153
2         285.999939
3         285.999939
4         285.999939
             ...    
314292    285.999939
314293     11.999331
314294    285.999939
314295    445.000244
314296    285.999939
Length: 314297, dtype: float32
Cálculo modelo de entrenamiento algoritmo Regresion Lineal de cuML = 0.007107973098754883


In [31]:
#Fase Uso de los algoritmos ML de cuml: Algoritmo Logistic Regression
import cupy as cp
from cuml.linear_model import LogisticRegression

#Creación de un nuevo dataframe Train 
dataTrainRe = cudf.DataFrame()
dataTrainRe['Usuarios'] = cp.asarray(value_usernameX, dtype=cp.float32)
dataTrainRe['Evento Fuente'] = cp.asarray(value_EventSourceX, dtype=cp.float32)

#Creación de un nuevo dataframe Test
new_valueRE = cp.asarray(value_usernameY, dtype=cp.float32)
dataTestRE = cudf.Series(new_valueRE)

startRlog = time.time()

#Entrenando el algoritmo
reg_lo = LogisticRegression(fit_intercept = True,
                            max_iter = 1000)
reg_lo.fit(dataTrainRe, dataTestRE)

endRlog = time.time()
calculatedRlog = endRlog - startRlog

print("Coefficients:")
print(reg_lo.coef_)
print("Intercept:")
print(reg_lo.intercept_)
print('Predicción del modelo:')
print(reg_lo.predict(dataTrain))

print("Cálculo modelo de entrenamiento algoritmo Regresión Logistica de cuML = {}".format(calculatedRlog))

[W] [19:31:02.162953] L-BFGS: max iterations reached
[W] [19:31:02.164421] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
Coefficients:
        0         1         2         3         4         5         6    \
0  0.756069  0.756718  0.759149  0.907988  1.044077  0.758952  0.760566   
1  1.030736  0.998777  0.999642  1.036844  1.159546  0.998777  0.998665   

        7         8         9    ...       667       668      669       670  \
0  0.759919  0.759919  0.762585  ...  1.055988  1.055974  1.05498  1.055354   
1  0.998777  0.998777  0.998665  ...  0.998053  0.998053  0.99319  0.992793   

        671       672       673       674       675       676  
0  1.071099  1.042548  1.064883  1.069374  1.069785  1.054973  
1  0.970061  0.996599  0.987109  0.961606  0.978600  0.998036  

[2 rows x 677 columns]
Intercept:
0      1.037304
1      0.997781
2      0.99

In [34]:
from cuxfilter import DataFrame, themes, layouts
from cuxfilter.assets.custom_tiles import get_provider, Vendors

inicioB = time.time()
#Creación DataFrame cuxfilter
cux_df = cuxfilter.DataFrame.from_dataframe(dataTrain)

#crear gráfico de barras estadísticas
bar_chart_1 = cuxfilter.charts.bar('day', 'username',
                                   title='Gráfico de Barras Estadístico',
                                   data_points=5, add_interaction=False)
#Crear el dashboard
d = cux_df.dashboard([bar_chart_1])
#vista previa del gráfico
bar_chart_1.view()

finB = time.time()
calculoB = finB - inicioB

print("Cálculo Visualización con cuxfilter gráfico de Barras = {}".format(calculoB))

Cálculo Visualización con cuxfilter gráfico de Barras = 0.6280198097229004


In [35]:
#crear gráfico de líneas estadisticas
inicioL = time.time()

line_chart_1 = cuxfilter.charts.line('day', 'username',
                                     title='Gráfico de Lineas Estadístico',
                                     data_points=5, add_interaction=False)
#Crear dashboard
dl = cux_df.dashboard([line_chart_1])
line_chart_1.view()

finL = time.time()
calculoL = finL - inicioL

print("Cálculo Visualización con cuxfilter gráfico de Líneas = {}".format(calculoL))

Cálculo Visualización con cuxfilter gráfico de Líneas = 0.6340129375457764
